# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Imports here
import random
import matplotlib.pyplot as plt

import numpy as np

import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

from PIL import Image

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [2]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [3]:
# TODO: Define your transforms for the training, validation, and testing sets

train_data_transform = transforms.Compose([
    transforms.RandomRotation(random.randint(0, 30)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

test_valid_data_transform = transforms.Compose(
    [transforms.Resize(256),
     transforms.CenterCrop(224),
     transforms.ToTensor(),
     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

# TODO: Load the datasets with ImageFolder
train_dataset = datasets.ImageFolder(train_dir, transform = train_data_transform)
valid_dataset = datasets.ImageFolder(valid_dir, transform = test_valid_data_transform)
test_dataset = datasets.ImageFolder(test_dir, transform = test_valid_data_transform)

# TODO: Using the image datasets and the trainforms, define the dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=64, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

data_iter = iter(train_dataloader)
images, labels = data_iter.next()
print(type(images))
print(images.shape)
print(labels.shape)

<class 'torch.Tensor'>
torch.Size([64, 3, 224, 224])
torch.Size([64])


### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [4]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
    print('Dataset has {} categories\n\n'.format(len(cat_to_name)),
          'List of labels:')
    cat_to_name = dict(sorted(cat_to_name.items(), key = lambda x:x[1]))
    i = 1
    for item in cat_to_name.items():
        print('{}. label: {}; index : {}'.format(i, item[1], item[0]))
        i += 1

Dataset has 102 categories

 List of labels:
1. label: alpine sea holly; index : 35
2. label: anthurium; index : 80
3. label: artichoke; index : 29
4. label: azalea; index : 72
5. label: ball moss; index : 93
6. label: balloon flower; index : 19
7. label: barbeton daisy; index : 41
8. label: bearded iris; index : 68
9. label: bee balm; index : 92
10. label: bird of paradise; index : 8
11. label: bishop of llandaff; index : 56
12. label: black-eyed susan; index : 63
13. label: blackberry lily; index : 102
14. label: blanket flower; index : 100
15. label: bolero deep blue; index : 45
16. label: bougainvillea; index : 95
17. label: bromelia; index : 99
18. label: buttercup; index : 48
19. label: californian poppy; index : 65
20. label: camellia; index : 96
21. label: canna lily; index : 90
22. label: canterbury bells; index : 3
23. label: cape flower; index : 37
24. label: carnation; index : 31
25. label: cautleya spicata; index : 61
26. label: clematis; index : 82
27. label: colt's foot;

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

<font color='red'>**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.</font>

In [5]:
# TODO: Build and train your network
my_model = models.vgg16(pretrained = True)
# freeze features for model
for param in my_model.parameters():
    param.requires_grad = False

In [6]:
# define calssifier architecture
my_model.classifier = nn.Sequential(nn.Linear(25088, 4096),
                                 nn.ReLU(),
                                 nn.Dropout(p=0.25, inplace=False),
                                 nn.Linear(4096, 512),
                                 nn.ReLU(),
                                 nn.Dropout(p=0.25, inplace=False),
                                 nn.Linear(512, 102),
                                 nn.LogSoftmax(dim = 1))
my_model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [7]:
# model training
def train_nn(dataloader, model, loss_fn, optimizer, device):
    
    # store running loss
    running_loss = 0
    # data container for tracking loss progress  
    train_losses = []
    
    # set model in training mode
    model.train()
    
    for images, labels in dataloader:
        
        # move computations
        images, labels = images.to(device), labels.to(device)

        # forward pass through the network
        pred = model(images)
        # calculate the loss
        loss = loss_fn(pred, labels)

        # clear the gradient
        optimizer.zero_grad()
        # perform a backward pass 
        loss.backward()
        # update the weights
        optimizer.step()

        # extract and accumulate loss value  
        running_loss += loss.item()
    
    return running_loss/len(dataloader)

In [8]:
# model testing
def test_nn(dataloader, model, loss_fn, device):
    
    # initialize variable to track model performance
    test_loss, accuracy = 0, 0
    test_losses = []
    # set model to evaluation mode 
    model.eval()
    
    # turn off gradient
    with torch.no_grad():
        for images, labels in dataloader:
            # move computations
            images, labels = images.to(device), labels.to(device)
            # forward pass through the network
            pred = model(images)
            
            # extract and accumulate loss value  
            test_loss += loss_fn(pred, labels).item()
            
            # get the class probabilities
            ps = torch.exp(model(images))
            # return top probabilietes and classes
            top_p, top_class = ps.topk(1, dim=1)
            # check number of matches for  pred / labels 
            equals = top_class == labels.view(*top_class.shape)
            accuracy  += torch.mean(equals.type(torch.FloatTensor)).item()
    
    return (test_loss/len(dataloader), accuracy/len(dataloader))

In [9]:
# train neural network
def train_nn_epoches(train_dataloader,
                     valid_dataloader,
                     model, 
                     criterion, 
                     optimizer,
                     model_performance):
    
    print('Neural network training has begun!\nBe patient it may take a while...\n')
    
    # check if GPU computation is available 
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print('Using {} device\n'.format(device))
    # move model to GPU or CPU
    model.to(device);
    
    epochs = model_performance['epoches']
    
    for epoch in range(epochs):

        train_loss = train_nn(train_dataloader, model, criterion, optimizer, device)
        valid_data = test_nn(valid_dataloader, model, criterion, device)

        (valid_loss, accuracy) = valid_data

        # store testing performance
        model_performance['train losses'].append(train_loss)
        model_performance['valid losses'].append(valid_loss)

        print('Epoch: {}/{};'.format(epoch+1, epochs),
              'Train loss: {:.3f};'.format(train_loss),
              'Validation loss: {:.3f};'.format(valid_loss),
              'Validation accuracy: {:.2f}%'.format(accuracy*100.0))

    print('Training done!') 

In [10]:
# training data
train_performance = {}
train_performance['epoches'] = 20
train_performance['train losses'] = []
train_performance['valid losses'] = []
print(train_performance)

{'epoches': 20, 'train losses': [], 'valid losses': []}


In [11]:
# loss function for optimizer 
# https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html?highlight=nllloss#torch.nn.NLLLoss
criterion = nn.NLLLoss()
# define optimizer and learning rate
optimizer = optim.Adam(my_model.classifier.parameters(), lr = 0.001)

In [ ]:
train_nn_epoches(train_dataloader,
                 valid_dataloader,
                 my_model,
                 criterion,
                 optimizer,
                 train_performance)

Neural network training has begun!
Be patient it may take a while...

Using cuda device

Epoch: 1/20; Train loss: 3.094; Validation loss: 1.143; Validation accuracy: 69.12%
Epoch: 2/20; Train loss: 1.667; Validation loss: 0.894; Validation accuracy: 74.98%
Epoch: 3/20; Train loss: 1.306; Validation loss: 0.643; Validation accuracy: 81.25%
Epoch: 4/20; Train loss: 1.157; Validation loss: 0.571; Validation accuracy: 85.34%
Epoch: 5/20; Train loss: 1.071; Validation loss: 0.540; Validation accuracy: 86.46%
Epoch: 6/20; Train loss: 1.006; Validation loss: 0.518; Validation accuracy: 85.39%
Epoch: 7/20; Train loss: 0.991; Validation loss: 0.506; Validation accuracy: 85.82%
Epoch: 8/20; Train loss: 0.915; Validation loss: 0.482; Validation accuracy: 88.02%
Epoch: 9/20; Train loss: 0.884; Validation loss: 0.426; Validation accuracy: 88.48%
Epoch: 10/20; Train loss: 0.853; Validation loss: 0.445; Validation accuracy: 88.51%
Epoch: 11/20; Train loss: 0.847; Validation loss: 0.453; Validation ac

In [ ]:
# plot results
plt.plot(train_performance['train losses'], label = 'Training loss')
plt.plot(train_performance['valid losses'], label = 'Validation loss')
plt.legend(frameon = False)
plt.xlabel('epochs')
plt.ylabel('negative log likelihood loss')
plt.show()

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
# TODO: Do validation on the test set
test_data = test_nn(test_dataloader,
                    my_model, 
                    criterion,
                    device = 'cuda' if torch.cuda.is_available() else 'cpu')

(_, accuracy) = test_data
print('Test accuracy: {:.2f}%'.format(accuracy*100.0))

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# TODO: Save the checkpoint 
# add mapping of indices to a model
def save_checkpoint(model,
                    optimizer,
                    dataset,
                    model_performance,
                    filepath = 'checkpoint.pth'):
    
    chceckpoint = {'epoches' : model_performance['epoches'],
                   'train losses' : model_performance['train losses'],
                   'valid losses' : model_performance['valid losses'],
                   'model state dict': model.state_dict(),
                   'optimizer state' : optimizer.state_dict(),
                   'classes to indices' : dataset.class_to_idx
                  }

    # save model state
    torch.save(chceckpoint, 'checkpoint.pth')
    print('Model saved successfully!')

In [ ]:
save_checkpoint(my_model, optimizer, train_dataset, train_performance)

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
# load a chceckpoint
def load_checkpoint(model, 
                    optimizer, 
                    filepath = 'checkpoint.pth',
                    print_state = False):
    
    '''Load: model performance, model data and optimizer state from file'''
    
    if torch.cuda.is_available():
        state_dict = torch.load(filepath)
    else:
        state_dict = torch.load(filepath, map_location='cpu')
    
    # load chceckpoint data
    model_performance = {}
    model_performance['epoches'] = state_dict['epoches']
    model_performance['train losses'] = state_dict['train losses']
    model_performance['valid losses'] = state_dict['valid losses']
    
    model.load_state_dict(state_dict['model state dict'],  strict=False)
    optimizer.load_state_dict(state_dict['optimizer state'])
    
    class_to_idx = state_dict['classes to indices']
    
    # print state dict
    if print_state:
        for i in state_dict.items():
            print(i, '\n')
    
    print('Data has been successfully loaded')
    return class_to_idx, model_performance

In [ ]:
class_to_idx, train_performance = load_checkpoint(my_model, optimizer)

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    
    # TODO: Process a PIL image for use in a PyTorch model
    with Image.open(image) as im:
        im_transform = transforms.Compose([transforms.Resize(256),
                                           transforms.CenterCrop(224),
                                           transforms.ToTensor(),
                                           transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
        
        im_tensor = torch.transpose(im_transform(im), 1, 1)
        return np.array(im_tensor)

In [ ]:
sample_image = process_image('flowers/test/102/image_08015.jpg')
print(np.shape(sample_image))

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    #image = image.numpy().transpose((1, 2, 0))
    image = image.transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = (std * image) + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    #ax.rotate_deg(90)
    ax.imshow(image)
    
    return ax

In [ ]:
imshow(sample_image)

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file
    # import and convert image to tensor
    image = torch.tensor(process_image(image_path))
    # adjust tensor dimentions
    image = torch.unsqueeze(image, 0)
    # move tensor to cpu or gpu
    image = image.to(device = 'cuda' if torch.cuda.is_available() else 'cpu')
    with torch.no_grad():
        # set model in evaluation mode
        model.eval()
        # get model probabilities
        prob = torch.exp(model(image))
    # compute and return top probabilities and classes
    top_p, top_class = prob.topk(topk, dim=1)
    
    return tuple(np.array(top_p).tolist()[0]), tuple(np.array(top_class).tolist()[0])

In [ ]:
top_p, top_class = predict('flowers/test/102/image_08015.jpg', my_model)
print(top_p, top_class)

In [ ]:
idx_to_class = {i:k for k, i in class_to_idx.items()}
# labels mapping
#print(idx_to_class,'\n')
#print(train_dataset.class_to_idx)
#print(cat_to_name, '\n')
#print(cat_to_name[idx_to_class[top_class[0]]])

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes

#test_image_path = 'flowers/test/102/image_08015.jpg'
test_image_path = 'flowers/test/69/image_05959.jpg'
#test_image_path = 'flowers/test/18/image_04292.jpg'
#test_image_path = 'flowers/test/7/image_07219.jpg'

test_image = process_image(test_image_path)
test_top_p, test_top_class = predict(test_image_path, my_model)

# get top model labels
flowers_labels = [cat_to_name[idx_to_class[test_top_class[i]]] for i in range(len(test_top_class))]
#print(flowers_labels)

# plot image and top model classes
fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(8,8))
fig.tight_layout()

ax1.set_title(cat_to_name[idx_to_class[test_top_class[0]]])
ax1.axis('off')
imshow(test_image, ax1)

y_pos = np.arange(len(test_top_class))
ax2.barh(y_pos, test_top_p)
ax2.set_yticks(y_pos)
ax2.set_yticklabels(flowers_labels)
ax2.set_aspect(0.21)
ax2.set_title('Class Probability')
ax2.set_xlim(0, 1.1)
ax2.invert_yaxis() 
plt.show()

<font color='red'>**Reminder for Workspace users:** If your network becomes very large when saved as a checkpoint, there might be issues with saving backups in your workspace. You should reduce the size of your hidden layers and train again. 
    
We strongly encourage you to delete these large interim files and directories before navigating to another page or closing the browser tab.</font>

In [ ]:
# TODO remove .pth files or move it to a temporary `~/opt` directory in this Workspace